In [ ]:
import tensorflow as tf
from keras.applications import MobileNet
import pathlib
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input,Dense, Flatten
from keras.models import Model
import matplotlib.pyplot as plt

In [2]:
input_shape = [224, 224]

In [3]:
train_path = '/home/user/Documents/augmanted_data/train'
val_path = '/home/user/Documents/augmanted_data/test'

In [4]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [ ]:
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 64,
                                                 class_mode = 'categorical',)

In [6]:
# val_set = train_datagen.flow_from_directory(val_path,
#                                                  target_size = (224, 224),
#                                                  batch_size = 64,
#                                                  class_mode = 'categorical')

In [ ]:
base_model = MobileNet(input_shape=input_shape + [3], weights='imagenet', include_top=False)

In [8]:
avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(10, activation='softmax')(avg)
model = tf.keras.Model(inputs=base_model.input, outputs=output)

In [9]:
base_model.trainable = False

In [ ]:
model.summary()

In [11]:
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.05)

In [12]:
model.compile(optimizer='Adam',loss="categorical_crossentropy",metrics="accuracy")

In [ ]:
network = model.fit(training_set, epochs=30, validation_data=training_set)

In [16]:
acc = network.history['accuracy']
val_acc = network.history['val_accuracy']

loss = network.history['loss']
val_loss = network.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [15]:
# model.save('my_model.h5')

In [26]:
# load = tf.keras.models.load_model('/home/user/Documents/Python_projects/classification_model/my_model.h5')